<a href="https://colab.research.google.com/github/thisissamuca/GOES_16/blob/main/GET_MASK_HIDRICA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Usado para obter recorte de corpos hídricos

In [ ]:
pip install rasterio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 87.3 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
import rasterio
import numpy as np
from tqdm import tqdm

Mounted at /content/drive


# Apenas um arquivo

In [ ]:
def reclassify_raster(input_path, output_path):
    """
    Reclassifica um raster GeoTIFF em duas etapas:
    1. Valores < 0.08 são reclassificados para 0
    2. Valores > 0 são reclassificados para 1

    Args:
        input_path (str): Caminho para o arquivo GeoTIFF de entrada
        output_path (str): Caminho para salvar o arquivo reclassificado
    """
    try:
        with rasterio.open(input_path) as src:
            # Ler os dados do raster
            data = src.read(1)
            meta = src.meta.copy()

            # Primeira reclassificação: valores < 0.08 -> 0
            data[data < 0.08] = np.nan

            # Segunda reclassificação: valores > 0 -> 1
            data[data > 0] = 1

            # Atualizar metadados para refletir as mudanças
            meta.update({
                'dtype': 'float32',  # ou 'uint8' se preferir valores inteiros
                'nodata': 0          # definir 0 como valor nodata (opcional)
            })

            # Salvar o raster reclassificado
            with rasterio.open(output_path, 'w', **meta) as dst:
                dst.write(data.astype(meta['dtype']), 1)

        print(f"Raster reclassificado com sucesso e salvo em: {output_path}")

    except Exception as e:
        print(f"Erro ao processar o arquivo: {e}")

# Exemplo de uso:
input_file = "/content/clip_ABI-L2-CMIPF-M6C03_G16_s20201521350164_e20201521359471_c20201521359550.tif"
output_file = "/content/teste.tif"
reclassify_raster(input_file, output_file)

Raster reclassificado com sucesso e salvo em: /content/teste.tif


# Lote de arquivos

In [ ]:
def batch_reclassify(input_dir, output_dir):
    """
    Processa todos os GeoTIFFs contendo 'M6C03' no nome em um diretório
    com as reclassificações especificadas

    Args:
        input_dir (str): Diretório com os arquivos de entrada
        output_dir (str): Diretório para salvar os arquivos processados
    """
    # Criar diretório de saída se não existir
    os.makedirs(output_dir, exist_ok=True)

    # Listar arquivos .tif no diretório que contenham 'M6C03' no nome
    tif_files = [f for f in os.listdir(input_dir)
                if f.lower().endswith('.tif') and 'M6C03' in f]

    if not tif_files:
        print("Nenhum arquivo .tif com 'M6C03' no nome encontrado no diretório de entrada.")
        return

    print(f"Processando {len(tif_files)} arquivos com 'M6C03'...")

    for file in tqdm(tif_files, desc="Processando arquivos M6C03"):
        try:
            input_path = os.path.join(input_dir, file)
            output_path = os.path.join(output_dir, f"reclass_{file}")

            with rasterio.open(input_path) as src:
                data = src.read(1)
                meta = src.meta.copy()

                # Primeira reclassificação: valores < 0.08 -> 0
                data[data < 0.08] = 0

                # Segunda reclassificação: valores > 0 -> 1
                data[data > 0] = 1

                # Atualizar metadados
                meta.update({
                    'dtype': 'float32',  # Usando inteiro para economizar espaço
                    'nodata': 0
                })

                with rasterio.open(output_path, 'w', **meta) as dst:
                    dst.write(data.astype(meta['dtype']), 1)

        except Exception as e:
            print(f"\nErro ao processar {file}: {e}")

    print("\nProcessamento concluído!")

# Exemplo de uso:
batch_reclassify("/content/drive/MyDrive/NBR/ARQUIVOS_PROCESSADOS/RECORTE_DADOS", "/content/drive/MyDrive/NBR/ARQUIVOS_PROCESSADOS/MASCARA_HÍDRICA")

Processando 12 arquivos com 'M6C03'...


Processando arquivos M6C03: 100%|██████████| 12/12 [00:15<00:00,  1.28s/it]


Processamento concluído!
